#### Importações

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV


#### Carrega e pré processa os dados

In [152]:
# Column names for the dataset
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

# Load the Adult dataset
train_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
test_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"

# Load training data
train_data = pd.read_csv(train_url, header=None, names=columns, na_values=["?", 99999], skipinitialspace=True)

# Load test data
test_data = pd.read_csv(test_url, header=None, names=columns, na_values=["?", 99999], skipinitialspace=True, skiprows=1)
test_data["income"] = test_data["income"].str.strip(".")  # Clean income column in test set

# Combine train and test data for consistent preprocessing
data = pd.concat([train_data, test_data], axis=0, ignore_index=True)

# Encode the target variable
label_encoder = LabelEncoder()
data["income"] = label_encoder.fit_transform(data["income"])  # Encodes '<=50K' as 0 and '>50K' as 1

# Remove the 'fnlwgt' column
data = data.drop(columns=['fnlwgt', 'education-num'])

# # Combinar categorias da variável 'workclass'
# data['workclass'] = data['workclass'].replace(
#     {'Without-pay': 'Without-pay', 'Never-worked': 'Without-pay'}
# )

# # Combinar categorias da variável 'education'
# data['education'] = data['education'].replace({
#     '11th': 'School-level', '9th': 'School-level', '7th-8th': 'School-level',
#     '5th-6th': 'School-level', '10th': 'School-level', '1st-4th': 'School-level',
#     'Preschool': 'School-level',
#     'Masters': 'Higher-Ed', 'Doctorate': 'Higher-Ed'
# })

# # Combinar categorias da variável 'marital-status'
# data['marital-status'] = data['marital-status'].replace({
#     'Married-civ-spouse': 'Married',
#     'Married-spouse-absent': 'Married',
#     'Married-AF-spouse': 'Married'
# })
# data['marital-status'] = data['marital-status'].replace({
#     'Divorced': 'Not-Married', 'Never-married': 'Not-Married',
#     'Separated': 'Not-Married', 'Widowed': 'Not-Married'
# })

# # Combinar categorias da variável 'race'
# data['race'] = data['race'].replace({
#     'Asian-Pac-Islander': 'Other-Race', 
#     'Amer-Indian-Eskimo': 'Other-Race', 
#     'Other': 'Other-Race'
# })

# Combinar categorias da variável 'native-country'
data['native-country'] = data['native-country'].replace({
    country: 'Other' for country in data['native-country'].unique() if country != 'United-States'
})


#### Transformações para modelagem

In [162]:
# Split features and target
X = data.drop(columns=["income"])
y = data["income"]

# Label Encoding para education, sex e race
for col in ["education", "sex", "native-country"]:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Define categorical and numerical columns
categorical_columns = X.select_dtypes(include=["object"]).columns
numerical_columns = X.select_dtypes(include=["int64", "float64"]).columns

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_columns),  # Scale numerical features
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_columns),  # One-hot encode categorical features
    ],
    remainder="passthrough"
)

# Split back into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=len(test_data))

#### Pipeline para Regressão Logística

In [11]:
# Create a pipeline with preprocessing and classifier
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000)),
])

#### Pipeline para Random Forest

In [13]:
# Create a pipeline with preprocessing and Random Forest classifier
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)),
])

#### Treina o modelo

In [14]:
# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.814937657392052

Classification Report:
               precision    recall  f1-score   support

       <=50K       0.86      0.90      0.88     12327
        >50K       0.64      0.56      0.59      3954

    accuracy                           0.81     16281
   macro avg       0.75      0.73      0.74     16281
weighted avg       0.81      0.81      0.81     16281



#### Hyperparameter tunning Random Forest

In [ ]:
# Define the parameter grid
param_grid = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [None, 10, 20, 30, 40],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None],
    "bootstrap": [True, False],
}

# Create the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Use RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=50,  # Number of random combinations to try
    scoring="accuracy",
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all available processors
)

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Fit the model
random_search.fit(X_train_preprocessed, y_train)

# Best parameters
print("Best Parameters:", random_search.best_params_)

# Evaluate the best model
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_preprocessed)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

#### Treina XGBoost

In [163]:
# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Train the XGBoost model directly
model = XGBClassifier(eval_metric="logloss", n_jobs=-1)
model.fit(X_train_preprocessed, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_preprocessed)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test_preprocessed)[:, 1]))


Accuracy: 0.8712609790553406

Classification Report:
               precision    recall  f1-score   support

       <=50K       0.90      0.94      0.92     12420
        >50K       0.77      0.65      0.71      3861

    accuracy                           0.87     16281
   macro avg       0.83      0.80      0.81     16281
weighted avg       0.87      0.87      0.87     16281

ROC-AUC: 0.9265443151111428


#### Hyperparameter tunning XGBoost

In [ ]:
# Define the parameter grid
param_grid = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [3, 5, 7, 10],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "gamma": [0, 0.1, 0.2, 0.5],
    "reg_alpha": [0, 0.01, 0.1, 1],
    "reg_lambda": [0.1, 1, 10]
}

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

class SklearnXGBClassifier(XGBClassifier):
    def __sklearn_tags__(self):
        return {}

# Create the model
xgb = SklearnXGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)

# Use RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=50,  # Number of random combinations to try
    scoring="accuracy",
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all processors
)


# Fit RandomizedSearchCV on training data
random_search.fit(X_train_preprocessed, y_train)

# Best parameters and model evaluation
print("Best Parameters:", random_search.best_params_)

# Make predictions
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_preprocessed)

# Evaluate the performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
